In [ ]:


# Initialize variables

def fplot(pid):

    pid_path = f'/mnt/data/AdaptiveControl/IBLrawdata/pid_data/{pid}'
    versions = ['', '_csd', '_bipolar']
    path_epochs = os.path.join(pid_path, f'lfp_{pid}_epoched.fif')
    epoch = mne.read_epochs(path_epochs, preload=True)
    row = len(epoch.ch_names)
    # Prepare figure
    fig, axes = plt.subplots(len(epoch.ch_names), len(versions), figsize=(15, 2*len(epoch.ch_names)), sharex=True)

    for col, version in enumerate(versions):
        # Read epochs for each version
        path_epochs = os.path.join(pid_path, f'lfp_{pid}_epoched{version}.fif')
        epoch = mne.read_epochs(path_epochs, preload=True)
        meta = epoch.metadata.reset_index()
        skewness_high = meta.index[meta['skewness'] > 1.5].tolist()
        # Extract indices and filter out those present in skewness_high
        high_index = [i for i in meta.index[(meta['contrastLeft'] == 1) | (meta['contrastRight'] == 1)] if i not in skewness_high]
        low_index = [i for i in meta.index[(meta['contrastLeft'] < 0.1) | (meta['contrastRight'] < 0.1)] if i not in skewness_high]

        epoch_high = epoch[high_index]
        epoch_low = epoch[low_index]
        evoked_high = epoch_high.average().crop(tmin=-0.2, tmax=0.4)
        evoked_low = epoch_low.average().crop(tmin=-0.2, tmax=0.4)
        evoked_high_data = evoked_high.data
        evoked_low_data = evoked_low.data
        
        # Get the channel names
        ch_names = epoch.ch_names

        # Iterate through each channel and plot the evoked responses for right and left trials
        for i, ch_name in enumerate(ch_names):
            ax = axes[i, col]
            ax.plot(evoked_high.times, evoked_high_data[i], label='high contrast Trial', color='blue')
            ax.plot(evoked_low.times, evoked_low_data[i], label='low contrast', color='red')
            ax.set_title(f'{ch_name} - {version}')
            ax.axvline(x=0, linestyle='--', color='k')  # Mark the stimulus onset
            if i == 0:
                ax.legend()
            if i == len(ch_names) - 1:
                ax.set_xlabel('Time (s)')
            if col == 0:
                ax.set_ylabel('Amplitude')

    # Adjust layout
    plt.tight_layout()
    plt.savefig('stimVSnonstim.png')
    


In [ ]:
def plot_stimNO_stim(pid):
    try:

        pid_path = f'/mnt/data/AdaptiveControl/IBLrawdata/pid_data/{pid}'
        path_epochs = os.path.join(pid_path, f'lfp_{pid}_epoched.fif')
        epoch = mne.read_epochs(path_epochs, preload=True)
        row = math.ceil(len(epoch.ch_names) /5) 

        # # Read metadata
        meta = epoch.metadata.reset_index()
        skewness_high = meta.index[meta['skewness'] > 1.5].tolist()
        high_index = [i for i in meta.index[(meta['contrastLeft'] == 1) | (meta['contrastRight'] == 1)] if i not in skewness_high]
        low_index = [i for i in meta.index[(meta['contrastLeft'] < 0.1) | (meta['contrastRight'] < 0.1)] if i not in skewness_high]

        epoch_high = epoch[high_index]
        epoch_low = epoch[low_index]
        evoked_high = epoch_high.average().crop(tmin=-0.2, tmax=0.4)
        evoked_low = epoch_low.average().crop(tmin=-0.2, tmax=0.4)
        del epoch_high, epoch_low
        evoked_high_data = evoked_high.data
        evoked_low_data = evoked_low.data


        # Prepare figure
        fig, axes = plt.subplots(row, 5, figsize=(20, 2 * row))
        for i in range(row):
            for j in range(5):
                idx = i * 5 + j
                if idx < len(epoch.ch_names):
                    ch_name = epoch.ch_names[idx]
                    ax = axes[i, j]
                    ax.plot(evoked_low.times, evoked_low_data[idx], label='low contrast Trial', color='blue')
                    ax.plot(evoked_high.times, evoked_high_data[idx], label='high contrast Trial', color='red')
                    ax.set_title(f'{ch_name}')
                    ax.axvline(x=0, linestyle='--', color='k')  # Mark the stimulus onset
                    if i == 0 and j == 0:
                        ax.legend()
                    if i == row - 1:
                        ax.set_xlabel('Time (s)')
                    if j == 0:
                        ax.set_ylabel('Amplitude')
                else:
                    continue

        # Adjust layout
        plt.tight_layout()
        plt.show()
        plt.title(f'pid = {pid}')
        dir_save = '/crnldata/cophy/TeamProjects/mohammad/ibl-oscillations/_analyses/_IBLworkflows/stimVSnoStim/figures'
        os.makedirs(dir_save, exist_ok=True)
        plt.savefig(f'{dir_save}/{pid}.png')
    except Exception as e:
        print(f'Error: {e}')
        pass

In [ ]:
import pickle
import mne
import numpy as np
import os
import pandas as pd
import pickle
import submitit
import matplotlib.pyplot as plt
import math

print('#### Start submitting jobs #####')


for i, pid in enumerate(os.listdir('/mnt/data/AdaptiveControl/IBLrawdata/pid_data')):
    
    # executor is the submission interface (logs are dumped in the folder)
    executor = submitit.AutoExecutor(folder=os.getcwd()+'/logs/')
    executor.update_parameters(mem_gb=20, timeout_min=200, slurm_partition="CPU", cpus_per_task=1, slurm_comment='plot_evoked_rightVS_left')
    job = executor.submit(plot_stimNO_stim, pid)


In [ ]:
pid = 'b40d8c65-7ee8-4069-8b64-1a3f7907742f'
# executor is the submission interface (logs are dumped in the folder)
executor = submitit.AutoExecutor(folder=os.getcwd()+'/logs/')
executor.update_parameters(mem_gb=20, timeout_min=200, slurm_partition="CPU", cpus_per_task=1, slurm_comment='plot_evoked_rightVS_left')
job = executor.submit(plot_stimNO_stim, pid)

In [ ]:
pid = 'b40d8c65-7ee8-4069-8b64-1a3f7907742f'
# executor is the submission interface (logs are dumped in the folder)
executor = submitit.AutoExecutor(folder=os.getcwd()+'/logs/')
executor.update_parameters(mem_gb=20, timeout_min=200, slurm_partition="CPU", cpus_per_task=1, slurm_comment='plot_evoked_rightVS_left')
job = executor.submit(fplot, pid)